# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [5]:
import numpy as np
import pandas as pd
import math
from scipy import stats
import requests

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [6]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [17]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [19]:
peRatio = data['peRatio']
price = data['latestPrice']
data

{'avgTotalVolume': 107662382,
 'calculationPrice': 'close',
 'change': 0.34,
 'changePercent': 0.00247,
 'close': 142.48,
 'closeSource': 'ifocaifl',
 'closeTime': 1688694924004,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 141.88,
 'delayedPriceTime': 1736038215750,
 'extendedChange': 0.33,
 'extendedChangePercent': 0.00231,
 'extendedPrice': 142.5,
 'extendedPriceTime': 1687745906265,
 'high': 146.36,
 'highSource': 'cep ityidme5eunlar1de  ',
 'highTime': 1712249441294,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 141.186,
 'iexCloseTime': 1700936771268,
 'iexLastUpdated': 1731172679518,
 'iexMarketPercent': 0.01819684993158407,
 'iexOpen': 144.37,
 'iexOpenTime': 1678967688827,
 'iexRealtimePrice': 142.45,
 'iexRealtimeSize': 105,
 'iexVolume': 1394613,
 'lastTradeTime': 1725626072783,
 'latestPrice': 141.26,
 'latestSource': 'Close',
 'latestTime': 'October 10, 2022',
 'latestUpdate': 1688912472538,
 'latestVolume': 784

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [3]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [37]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if(symbol in data and data[symbol]['quote']['peRatio'] != None and data[symbol]['quote']['latestPrice'] != None):
            final_dataframe = final_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        data[symbol]['quote']['latestPrice'],
                        data[symbol]['quote']['peRatio'],
                        'N/A',
                    ], index = my_columns
                ), ignore_index=True
            )
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,130.29,29.87,N/A
1,AAL,12.46,-4.24,N/A
2,AAP,173.79,19.47,N/A
3,AAPL,141.58,24.18,N/A
4,ABBV,142.21,20.04,N/A
...,...,...,...,...
486,YUM,108.77,21.86,N/A
487,ZBH,107.03,103.02,N/A
488,ZBRA,263.70,28.46,N/A
489,ZION,50.40,5.84,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [41]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending=True, inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True, drop=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,41.50,2.98,N/A
1,AIG,49.89,3.33,N/A
2,PVH,47.50,3.59,N/A
3,NUE,121.57,3.71,N/A
4,F,11.65,4.05,N/A
5,COF,96.24,4.07,N/A
6,HPE,12.30,4.40,N/A
7,DISH,14.43,4.43,N/A
8,HPQ,26.13,4.44,N/A
9,PHM,40.95,4.46,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [4]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [56]:
portfolio_input()

Enter the value of your portfolio:1000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [59]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
# final_dataframe
# sum(final_dataframe['Number of Shares to Buy'])

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,41.50,2.98,0
1,AIG,49.89,3.33,0
2,PVH,47.50,3.59,0
3,NUE,121.57,3.71,0
4,F,11.65,4.05,1
5,COF,96.24,4.07,0
6,HPE,12.30,4.40,1
7,DISH,14.43,4.43,1
8,HPQ,26.13,4.44,0
9,PHM,40.95,4.46,0


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [64]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']
# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']
# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
ev_to_ebitda = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['EBITDA']
# Enterprise Value divided by Gross Profit (EV/GP)
ev_to_gross_profit = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['grossProfit']

13.643832344342606

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [70]:
rv_columns =[
    'Ticker', 
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score',
]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if(symbol in data):
            # Price-to-earnings ratio
            pe_ratio = data[symbol]['quote']['peRatio']
            # Price-to-book ratio
            pb_ratio = data[symbol]['advanced-stats']['priceToBook']
            # Price-to-sales ratio
            ps_ratio = data[symbol]['advanced-stats']['priceToSales']
            try:
                # Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
                ev_to_ebitda = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['EBITDA']
            except TypeError:
                ev_to_ebitda = np.NaN
            try:
                # Enterprise Value divided by Gross Profit (EV/GP)
                ev_to_gross_profit = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['grossProfit']
            except TypeError:
                ev_to_gross_profit = np.NaN
                
            rv_dataframe = rv_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        data[symbol]['quote']['latestPrice'],
                        'N/A',
                        pe_ratio,
                        'N/A',
                        pb_ratio,
                        'N/A',
                        ps_ratio,
                        'N/A',
                        ev_to_ebitda,
                        'N/A',
                        ev_to_gross_profit,
                        'N/A',
                        'N/A',
                    ], index = rv_columns
                ), ignore_index=True
            )
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128.93,N/A,28.93,N/A,7.49,N/A,5.82,N/A,21.282381,N/A,10.924367,N/A,N/A
1,AAL,12.41,N/A,-4.22,N/A,-0.9537,N/A,0.1975,N/A,108.331047,N/A,0.792512,N/A,N/A
2,AAP,171.44,N/A,19.73,N/A,3.55,N/A,0.903,N/A,10.727876,N/A,2.229364,N/A,N/A
3,AAPL,141.80,N/A,23.5,N/A,40.28,N/A,6.07,N/A,18.434955,N/A,14.109607,N/A,N/A
4,ABBV,142.24,N/A,20.38,N/A,17.09,N/A,4.46,N/A,11.529781,N/A,7.944346,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,112.14,N/A,22.04,N/A,-3.71,N/A,4.68,N/A,19.529616,N/A,8.669547,N/A,N/A
497,ZBH,106.98,N/A,101.94,N/A,1.92,N/A,3.11,N/A,17.800520,N/A,5.381294,N/A,N/A
498,ZBRA,264.20,N/A,28.52,N/A,5.6,N/A,2.49,N/A,14.267731,N/A,6.003638,N/A,N/A
499,ZION,50.08,N/A,5.72,N/A,1.49,N/A,2.26,N/A,4.578448,N/A,2.087246,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [71]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,189.200,N/A,60.75,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,289.880,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,274.270,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,99.130,N/A,48.34,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.18,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
121,CTXS,108.200,N/A,41.79,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,25.070,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,50.020,N/A,14.65,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,57.550,N/A,32.44,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,29.170,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [76]:
for column in [
    'Price',
    'Price-to-Earnings Ratio',
    'Price-to-Book Ratio',
    'Price-to-Sales Ratio',
    'EV/EBITDA',
    'EV/GP',
]:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [77]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [80]:
metrics = {
    'Price-to-Earnings Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile'
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] \
            = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128.93,N/A,28.93,74.850299,7.4900,78.043912,5.8200,84.231537,21.282381,86.626747,10.924367,83.433134,N/A
1,AAL,12.41,N/A,-4.22,3.393214,-0.9537,5.988024,0.1975,0.998004,108.331047,99.401198,0.792512,3.193613,N/A
2,AAP,171.44,N/A,19.73,50.898204,3.5500,55.08982,0.9030,15.369261,10.727876,36.926148,2.229364,14.570858,N/A
3,AAPL,141.80,N/A,23.50,61.776447,40.2800,97.804391,6.0700,85.928144,18.434955,76.846307,14.109607,91.616766,N/A
4,ABBV,142.24,N/A,20.38,52.894212,17.0900,93.612774,4.4600,76.946108,11.529781,41.117764,7.944346,67.065868,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,112.14,N/A,22.04,57.285429,-3.7100,5.588822,4.6800,78.443114,19.529616,81.237525,8.669547,72.854291,N/A
497,ZBH,106.98,N/A,101.94,97.804391,1.9200,33.932136,3.1100,58.483034,17.800520,75.0499,5.381294,42.914172,N/A
498,ZBRA,264.20,N/A,28.52,73.253493,5.6000,70.259481,2.4900,48.902196,14.267731,57.285429,6.003638,48.303393,N/A
499,ZION,50.08,N/A,5.72,9.580838,1.4900,22.355289,2.2600,44.211577,4.578448,9.181637,2.087246,13.173653,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [82]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128.93,N/A,28.93,74.850299,7.4900,78.043912,5.8200,84.231537,21.282381,86.626747,10.924367,83.433134,81.437126
1,AAL,12.41,N/A,-4.22,3.393214,-0.9537,5.988024,0.1975,0.998004,108.331047,99.401198,0.792512,3.193613,22.59481
2,AAP,171.44,N/A,19.73,50.898204,3.5500,55.08982,0.9030,15.369261,10.727876,36.926148,2.229364,14.570858,34.570858
3,AAPL,141.80,N/A,23.50,61.776447,40.2800,97.804391,6.0700,85.928144,18.434955,76.846307,14.109607,91.616766,82.794411
4,ABBV,142.24,N/A,20.38,52.894212,17.0900,93.612774,4.4600,76.946108,11.529781,41.117764,7.944346,67.065868,66.327345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,112.14,N/A,22.04,57.285429,-3.7100,5.588822,4.6800,78.443114,19.529616,81.237525,8.669547,72.854291,59.081836
497,ZBH,106.98,N/A,101.94,97.804391,1.9200,33.932136,3.1100,58.483034,17.800520,75.0499,5.381294,42.914172,61.636727
498,ZBRA,264.20,N/A,28.52,73.253493,5.6000,70.259481,2.4900,48.902196,14.267731,57.285429,6.003638,48.303393,59.600798
499,ZION,50.08,N/A,5.72,9.580838,1.4900,22.355289,2.2600,44.211577,4.578448,9.181637,2.087246,13.173653,19.700599


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [86]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe.reset_index(inplace=True, drop=True)
rv_dataframe = rv_dataframe[:50]

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [88]:
portfolio_input()

Enter the value of your portfolio:1000000


In [92]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
position_size
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
rv_dataframe.sort_values('Price')

/home/shahriar/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
16,CCL,6.56,3048,-1.07,4.391218,0.9248,10.578842,0.8115,13.373253,-13.023661,0.798403,3.423323,25.349301,10.898204
11,UAA,7.06,2832,9.31,18.163673,1.3500,18.562874,0.5129,7.185629,2.712640,2.195609,0.627871,1.397206,9.500998
28,GPS,10.02,1996,-9.57,2.794411,1.5900,24.251497,0.2318,1.596806,10.054581,32.934132,0.821625,3.592814,13.033932
4,F,11.47,1743,4.08,5.588822,1.0900,13.772455,0.3327,3.393214,3.229979,3.592814,1.683936,9.780439,7.225549
38,HPE,12.70,1574,4.50,6.586826,0.7799,8.582834,0.5730,8.782435,10.350820,34.730539,2.691905,18.762475,15.489022
21,DISH,14.41,1387,4.48,6.387226,0.2500,6.187625,0.2442,1.796407,6.646816,18.363273,3.768730,28.942116,12.335329
27,IVZ,15.03,1330,6.26,10.778443,0.4423,6.387226,1.0150,18.762475,6.592369,17.764471,1.877595,11.377246,13.013972
22,T,15.40,1298,5.76,9.780439,0.9223,10.179641,0.7173,11.576846,4.901557,10.379242,2.856355,21.157685,12.61477
46,NWSA,15.90,1257,15.49,38.722555,1.1000,13.972056,0.8970,14.670659,6.207177,15.768463,1.003853,5.389222,17.704591
30,BEN,22.06,906,6.56,12.175649,0.9869,11.377246,1.2900,25.349301,4.651493,9.580838,1.189204,6.986028,13.093812


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

## Saving Our Excel Output
As before, saving our Excel output is very easy: